Importy Bibloitek

In [ ]:
import json
import string

import itertools as it
import operator as op

import morfeusz2
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize

# Odkomentować przy odtwarzaniu środowiska Python
# import nltk
# nltk.download("punkt_tab")


1. Utworzenie korpusu dokumentów

In [6]:
corpus_dir = "./literatura"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files = corpus.fileids()

2. Wstepne przygotowanie dokumentów (preprocessing)

In [19]:
documents = dict()

for file_name in files:
    documents[file_name] = corpus.raw(file_name)

with open("./stopwords_pl.txt", "r", encoding="UTF-8") as stoplist_file:
    stoplist = stoplist_file.read()

stoplist = stoplist.splitlines()
stoplist = stoplist[4:]

def lemmatize(text: str):
    morf = morfeusz2.Morfeusz()
    segments = it.groupby(morf.analyse(text), op.itemgetter(0, 1))

    def disambiguate(group):
        pairs = ((len(descr), lemma) for _, _, (_, lemma,descr, _, _) in group)
        perpl, lemma = min(pairs)
        return lemma.split(":")[0]
    
    lemmas = (disambiguate(group) for key, group in segments)
    return " ".join(filter(str.isalpha, lemmas))

for key in documents:
    documents[key] = documents[key].lower()
    documents[key] = "".join([char for char in documents[key] if char not in string.punctuation])
    documents[key] = lemmatize(documents[key])
    documents[key] = "".join([word for word in word_tokenize(documents[key], language="polish") if word not in stoplist])
